# MSCF v2.1.7: Reproducing Key Results

This notebook reproduces the numerical results, figures, and tables from:

> Roland, I. (2026). *Matter-Space Coupling Framework: Deriving General Relativity as the IR Limit of Causal Substrate Dynamics* (MSCF v2.1.7).

All computations are self-contained. Heavy results (transfer function, phase correlations) are loaded from precomputed data in `results/`. Lightweight results (echo delays, ghost window) are computed inline.

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path

REPO_ROOT = Path('.').resolve().parent
RESULTS = REPO_ROOT / 'results'
FIGURES = REPO_ROOT / 'paper' / 'figures'

plt.rcParams.update({
    'font.size': 11,
    'font.family': 'serif',
    'axes.labelsize': 12,
    'figure.dpi': 150,
    'lines.linewidth': 1.5,
    'mathtext.fontset': 'cm',
})

---
## 1. Echo Delay Table (Table II, Section XI.D)

The echo delay is determined by the Kerr tortoise coordinate distance between the light ring and the MSCF barrier at $r_b = M$:

$$\Delta t_{\text{echo}}(M, \chi) = \frac{2}{c} |r_*(r_{\text{lr}}(\chi)) - r_*(M)|$$

This is a deterministic function of remnant mass and spin with no free parameters.

In [ ]:
def light_ring_radius(chi):
    """Co-rotating equatorial light ring radius in units of M."""
    chi = np.clip(chi, -1 + 1e-15, 1 - 1e-15)
    return 2 * (1 + np.cos(2/3 * np.arccos(-abs(chi))))


def kerr_rstar(r, chi):
    """Kerr tortoise coordinate r*(r) in units of M."""
    chi = np.clip(chi, -1 + 1e-15, 1 - 1e-15)
    s = np.sqrt(1 - chi**2)
    rp = 1 + s  # outer horizon
    rm = 1 - s  # inner horizon
    d = rp - rm
    if abs(d) < 1e-14:  # Schwarzschild limit
        return r + 2 * np.log(abs(r/2 - 1))
    Ap = 2 * rp / d
    Am = 2 * rm / d
    return r + Ap * np.log(abs((r - rp) / 2)) - Am * np.log(abs((r - rm) / 2))


def echo_delay_over_M(chi):
    """Echo delay dt/M = 2|r*(r_lr) - r*(r_b)| with r_b = M."""
    r_lr = light_ring_radius(chi)
    return 2 * abs(kerr_rstar(r_lr, chi) - kerr_rstar(1.0, chi))


# Table II values from the paper
chi_table = [0.0, 0.5, 0.9, 0.99, 0.999]

print('Table II: Predicted Echo Delays (MSCF v2.1.7, Section XI.D)')
print(f'{"chi":>6}  {"r_lr/M":>8}  {"dt/M_time":>10}')
print('-' * 30)
for chi in chi_table:
    r_lr = light_ring_radius(chi)
    dt = echo_delay_over_M(chi)
    print(f'{chi:6.3f}  {r_lr:8.3f}  {dt:10.3f}')

In [ ]:
# Plot echo delay vs spin
chi_fine = np.linspace(0, 0.999, 200)
dt_fine = [echo_delay_over_M(c) for c in chi_fine]

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(chi_fine, dt_fine, 'C0-', lw=2)
ax.set_xlabel(r'Spin $\chi$')
ax.set_ylabel(r'$\Delta t_{\rm echo}\, /\, M_{\rm time}$')
ax.set_title('Echo delay vs spin (MSCF v2.1.7, Eq. 35)')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 2. Transfer Function and Bogoliubov Phase (Figure 1, Section IX.E)

The coupled 2x2 mode evolution through the bounce produces a transfer function $T^2(k)$ and a Bogoliubov phase $\arg(\alpha_k)$. The bounce produces ~330 degrees of phase variation at $k > 10^{-2}$ (Planck units), but CMB modes at $k < 10^{-4}$ lie in the asymptotic plateau where both amplitude and phase are constant to < 0.2%.

Data loaded from precomputed results of `coupled_mode_evolution.py` and `phase_correlation_test.py`.

In [ ]:
# Load precomputed data
phase_dat = np.loadtxt(RESULTS / 'cmb_comparison' / 'phase_function.dat')
k_phase = phase_dat[:, 0]
arg_alpha = phase_dat[:, 1]

with open(RESULTS / 'cmb_comparison' / 'phase_results.json') as f:
    phase_json = json.load(f)

with open(RESULTS / 'transfer_functions' / 'full_coupled_results.json') as f:
    coupled_json = json.load(f)

k_coupled = np.array([m['k'] for m in coupled_json['coupled']])
T2_coupled = np.array([m['T2'] for m in coupled_json['coupled']])
k_uncoupled = np.array([m['k'] for m in coupled_json['uncoupled']])
T2_uncoupled = np.array([m['T2'] for m in coupled_json['uncoupled']])

print(f'Loaded {len(k_phase)} phase points, {len(k_coupled)} coupled T^2 points')
print(f'Low-k T^2 coupled/uncoupled ratio: {T2_coupled[0]/T2_uncoupled[0]:.3f} (32% suppression)')

In [ ]:
# Figure 1: Transfer function and Bogoliubov phase (4-panel summary)
unc_modes = phase_json.get('mode_results_uncoupled', [])
k_unc = np.array([m['k'] for m in unc_modes])
arg_alpha_unc = np.array([m['arg_alpha'] for m in unc_modes])
mp_data = phase_json['multipole_phases']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# (a) Phase across full k range
ax = axes[0, 0]
ax.plot(k_phase, arg_alpha, 'C0-', lw=2, label='Coupled')
ax.plot(k_unc, arg_alpha_unc, 'C1--', lw=1.5, label='Uncoupled')
ax.axvspan(1e-8, 1e-4, alpha=0.07, color='steelblue')
ax.axvspan(0.01, 10, alpha=0.07, color='firebrick')
ax.set_xscale('log')
ax.set_xlabel(r'$k$ [Planck units]')
ax.set_ylabel(r'$\arg(\alpha_k)$ [rad]')
ax.set_title(r'(a) Bogoliubov phase $\arg(\alpha_k)$')
ax.legend(loc='lower left', fontsize=8)
ax.set_xlim(1e-8, 20)
ax.set_ylim(-3.5, 2.8)
ax.grid(True, alpha=0.2)
ax.text(3e-7, 2.3, 'CMB scales', color='steelblue', fontsize=9, ha='center', fontweight='bold')
ax.text(0.3, 2.3, r'$\Xi$-well region', color='firebrick', fontsize=9, ha='center', fontweight='bold')
ax.annotate(r'$\Delta\phi \approx 330°$', xy=(0.1, -2.8), fontsize=9, color='firebrick', ha='center',
            bbox=dict(boxstyle='round,pad=0.2', fc='white', ec='firebrick', alpha=0.7))

# (b) CMB zoom
ax = axes[0, 1]
cmb_mask = k_phase <= 1e-4
ax.plot(k_phase[cmb_mask], arg_alpha[cmb_mask], 'C0-', lw=2.5, label='Coupled')
unc_cmb = k_unc <= 1e-4
ax.plot(k_unc[unc_cmb], arg_alpha_unc[unc_cmb], 'C1--', lw=1.8, label='Uncoupled')
ax.set_xscale('log')
ax.set_xlabel(r'$k$ [Planck units]')
ax.set_ylabel(r'$\arg(\alpha_k)$ [rad]')
ax.set_title(r'(b) CMB range: $\Delta\phi = 2.9\times10^{-3}$ rad')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.2)

# (c) Multipole phases
ax = axes[1, 0]
for kappa_str, label, color, ms in [
        ('0.000316', r'$\kappa = 3.16\times10^{-4}$', 'C0', 'o'),
        ('0.000562', r'$\kappa = 5.62\times10^{-4}$', 'C1', 's'),
        ('0.001', r'$\kappa = 1.00\times10^{-3}$', 'C2', '^')]:
    mp = mp_data[kappa_str]
    ells = sorted([int(e) for e in mp.keys()])
    phi_vals = np.array([mp[str(e)]['phi_arg_alpha'] for e in ells])
    phi_rel = phi_vals - phi_vals[0]
    ax.plot(ells, np.degrees(phi_rel) * 3600, f'{ms}-', markersize=3, label=label, color=color, lw=1.5)
ax.set_xlabel(r'Multipole $\ell$')
ax.set_ylabel(r'$\phi_\ell - \phi_{\ell=2}$ [arcsec]')
ax.set_title('(c) Multipole phase variation')
ax.legend(fontsize=8)
ax.set_xlim(1, 31)
ax.grid(True, alpha=0.2)

# (d) Coupling ratio
ax = axes[1, 1]
n = min(len(k_coupled), len(k_uncoupled))
ratio = T2_coupled[:n] / T2_uncoupled[:n]
ax.plot(k_coupled[:n], ratio, 'C3o-', ms=5, lw=2)
ax.axhline(1.0, color='gray', ls='--', lw=0.8)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel(r'$k$ [Planck units]')
ax.set_ylabel(r'$T^2_{\rm coupled} / T^2_{\rm uncoupled}$')
ax.set_title('(d) Landau-Zener coupling ratio')
ax.set_ylim(1e-3, 2)
ax.grid(True, alpha=0.2)
ax.annotate(f'Low-$k$ plateau: {ratio[0]:.3f}\n(32% suppression)',
            xy=(k_coupled[2], ratio[2]), xytext=(0.03, 0.12),
            arrowprops=dict(arrowstyle='->', color='C3', lw=1.2),
            fontsize=8, color='C3',
            bbox=dict(boxstyle='round,pad=0.3', fc='white', ec='C3', alpha=0.8))

fig.suptitle('Figure 1: Transfer function and Bogoliubov phase (MSCF v2.1.7)', fontsize=13, y=1.01)
fig.tight_layout()
plt.show()

---
## 3. Two-Surface Interior Structure (Figure 2, Sections X-XI)

The MSCF metric function $F(r) = (1 - x_g)(1 - x_g/2)$ has two zeros:
- The **horizon** at $r = r_s$ ($x_g = 1$): partial reflectivity $R_1$
- The **inversion barrier** at $r = M$ ($x_g = 2$): total reflectivity $R_2 = 1$

The cavity between them has proper distance $\Delta\ell \approx 1.285\, r_s$.

In [ ]:
from scipy.integrate import quad

def cavity_integrand(r_over_M):
    u = r_over_M
    F_abs = abs((2/u - 1) * (1 - 1/u))
    return 1 / np.sqrt(F_abs) if F_abs > 1e-14 else 0.0

delta_ell_M, _ = quad(cavity_integrand, 1.001, 1.999)
delta_ell_rs = delta_ell_M / 2
print(f'Cavity proper length: {delta_ell_M:.4f} M = {delta_ell_rs:.4f} r_s')

# Plot metric function
u = np.linspace(0.32, 3.5, 2000)
F_mscf = (1 - 1/u) * (1 - 1/(2*u))
f_gr = 1 - 1/u

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(u, F_mscf, 'k-', lw=2.5, label=r'MSCF: $F = (1-x_g)(1-x_g/2)$')
ax.plot(u, f_gr, color='#888', ls='--', lw=1.8, alpha=0.75, label=r'GR: $f = 1 - x_g$')
ax.axhline(0, color='black', lw=0.6)
ax.plot(1.0, 0, 'ko', ms=9, markeredgecolor='white', markeredgewidth=1.5)
ax.plot(0.5, 0, 'ks', ms=9, markeredgecolor='white', markeredgewidth=1.5)
ax.annotate('Horizon\n$r = r_s$ ($x_g = 1$)', xy=(1.0, 0), xytext=(1.55, -0.18),
            arrowprops=dict(arrowstyle='->', lw=1.2), fontsize=9, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', ec='#888', alpha=0.95))
ax.annotate('Inversion barrier\n$r = M$ ($x_g = 2$)', xy=(0.5, 0), xytext=(0.75, 0.32),
            arrowprops=dict(arrowstyle='->', lw=1.2), fontsize=9, ha='center',
            bbox=dict(boxstyle='round,pad=0.3', fc='lightyellow', ec='#888', alpha=0.95))
ax.set_xlabel(r'$r / r_s$')
ax.set_ylabel('Metric function')
ax.set_title('Figure 2: MSCF metric F(r) vs Schwarzschild f(r) (MSCF v2.1.7)')
ax.set_xlim(0.32, 3.5)
ax.set_ylim(-0.45, 0.65)
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.12)
plt.tight_layout()
plt.show()

---
## 4. Amplitude Inversion Theorem (Figure 3, Section XI.C)

**Theorem 11.1:** $A_2 > A_1$ if and only if $R_1 < \frac{3 - \sqrt{5}}{2} \approx 0.382$.

This is the binary discriminator between MSCF and all single-surface ECO models.

In [ ]:
R1_threshold = (3 - np.sqrt(5)) / 2
print(f'Amplitude inversion threshold: R1 < {R1_threshold:.6f}')
print(f'Current LIGO/Virgo constraint: R1 < 0.05 (90% CL)')
print(f'Since 0.05 < {R1_threshold:.3f}, MSCF predicts A2 > A1')

N_echoes = 8
n = np.arange(1, N_echoes + 1)

# Generic single-surface ECO: monotonic decay
R_generic = 0.65
A_generic = R_generic ** (n - 1)
A_generic = A_generic / A_generic[0]

# MSCF two-surface (Theorem 11.1: T = 1 - R1, amplitude convention)
R_horizon = 0.15
T_horizon = 1 - R_horizon  # amplitude transmission (paper Eq. 34)
R_barrier = 1.0             # total reflection at inversion barrier

A_mscf = np.zeros(N_echoes)
A_mscf[0] = R_horizon                      # A1: horizon reflection
A_mscf[1] = T_horizon**2 * R_barrier       # A2: transmit-reflect-transmit
for i in range(2, N_echoes):
    A_mscf[i] = A_mscf[1] * R_horizon ** (i - 1)
A_mscf = A_mscf / A_mscf.max()

fig, ax = plt.subplots(figsize=(8, 5.5))
ax.plot(n, A_generic, 'o--', color='#888', ms=10, lw=1.8,
        markeredgecolor='white', markeredgewidth=1.2,
        label='Generic ECO (single surface)')
ax.plot(n, A_mscf, 's-', color='#D94A4A', ms=11, lw=2.2,
        markeredgecolor='white', markeredgewidth=1.2,
        label='MSCF (two surfaces)')
ax.annotate('', xy=(2, A_mscf[1] + 0.02), xytext=(1, A_mscf[0] + 0.02),
            arrowprops=dict(arrowstyle='->', color='#A03030', lw=2.5, connectionstyle='arc3,rad=-0.3'))
ax.text(1.5, 0.52, r'$A_2 > A_1$' + '\ninversion', fontsize=12, ha='center',
        color='#A03030', fontweight='bold',
        bbox=dict(boxstyle='round,pad=0.3', fc='#FFF0F0', ec='#A03030', alpha=0.95, lw=1.5))
ax.set_xlabel('Echo number $n$')
ax.set_ylabel(r'Relative amplitude $A_n / A_{\max}$')
ax.set_title('Figure 3: Echo amplitude inversion (MSCF v2.1.7, Theorem 11.1)')
ax.set_xlim(0.4, N_echoes + 0.6)
ax.set_ylim(-0.05, 1.15)
ax.set_xticks(n)
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.15)
plt.tight_layout()
plt.show()

---
## 5. Ghost Window Analysis (Section IX.E.2)

The ghost window is where the matter eigenvalue $G_{\rm matt} < 0$. It is benign: sub-Planckian in duration, invisible to CMB modes.

In [ ]:
with open(RESULTS / 'ghost_analysis' / 'ghost_window.json') as f:
    ghost = json.load(f)

print('Ghost Window Analysis (MSCF v2.1.7, Theorem 9.13)')
print(f'  Duration (cosmic time):   {ghost["Delta_t_ghost_planck"]:.4f} t_Pl')
print(f'  Duration (conformal):     {ghost["Delta_eta_ghost_planck"]:.4f} eta_Pl')
print(f'  Resonance threshold:      k > {ghost["k_threshold"]:.4f} (Planck units)')
print(f'  CMB modes:                k * Delta_eta ~ {ghost["k_CMB_ratio"]}')
print(f'  Verdict:                  {ghost["verdict"]}')

---
## 6. Planck Commander Likelihood (Section IX.E.4)

Comparison against the Planck 2018 Commander TT likelihood at $\ell = 2$-$29$. LCDM is preferred at all tested values of the scale parameter $\kappa$.

In [ ]:
with open(RESULTS / 'cmb_comparison' / 'commander_results.json') as f:
    commander = json.load(f)

print('Planck Commander Likelihood (MSCF v2.1.7, Section IX.E.4)')
print(f'  logL(LCDM) = {commander["results"][0]["logL_lcdm"]:.2f}')
print()
for entry in commander['results']:
    kappa = entry['kappa']
    dlogL = entry['delta_logL']
    print(f'  kappa = {kappa:.2e}: Delta logL = {dlogL:.2f} (LCDM preferred)')

---
## 7. Algebraic Identities (Sections VI.E and IX.C)

Two algebraic results from the paper verified numerically:

**Theorem 6.4 (Self-Consistency):** $\xi / \eta^2 = 1$ for any $\ell_* / \ell_P$, where $\eta = \ell_*^2 / \ell_P^2$ (entropy per link) and $\xi = \ell_*^4 / \ell_P^4$ (max links per 4-cell).

**Theorem 9.8 (Substrate-Matter Unity):** $F_s^{\rm total} / M^2 = \varepsilon_m - 1$ where $\varepsilon_m = 3(1+w)/2$. Stability requires $w > -1/3$.

In [ ]:
# Theorem 6.4: Self-Consistency Identity
G = 6.67430e-11
c_light = 2.99792458e8
hbar = 1.054571817e-34
l_P = np.sqrt(hbar * G / c_light**3)
rho_crit = c_light**7 / (hbar * G**2)

print('Theorem 6.4: xi / eta^2 = 1 for all l_*')
print(f'  {"l_*/l_P":>10}  {"eta":>14}  {"xi":>14}  {"xi/eta^2":>16}')
print('  ' + '-' * 60)
for r in [0.37, 0.5, 1.0, np.sqrt(2), 2.0]:
    l_star = r * l_P
    eta = c_light**3 * l_star**2 / (hbar * G)
    xi = rho_crit * l_star**4 / (hbar * c_light)
    print(f'  {r:10.4f}  {eta:14.6e}  {xi:14.6e}  {xi / eta**2:16.10f}')

print()

# Theorem 9.8: Substrate-Matter Unity
print('Theorem 9.8: F_s^total / M^2 = eps_m - 1 = 3(1+w)/2 - 1')
print(f'  {"Matter":>14}  {"w":>6}  {"eps_m":>8}  {"F_s/M^2":>10}  {"Stable":>8}')
print('  ' + '-' * 52)
for name, w in [('Dust', 0.0), ('Radiation', 1/3), ('Stiff matter', 1.0), ('Dark energy', -1.0)]:
    eps_m = 3 * (1 + w) / 2
    fs = eps_m - 1
    print(f'  {name:>14}  {w:>6.2f}  {eps_m:>8.3f}  {fs:>+10.3f}  {"yes" if fs > 0 else "no":>8}')

---
## 8. Remnant Properties (Section XII, Theorems 12.2 and 12.4)

MSCF predicts a minimum black hole mass $M_{\rm min} = M_P / 2$ (Eq. 39) and a modified Hawking temperature $T_{\rm eff} = T_H [1 - \frac{1}{4}(M_P/M)^2]$ (Eq. 43) that vanishes at $M_{\rm min}$, yielding stable Planck-mass remnants.

The gravitational scattering cross-section of these remnants is $\sigma_n \sim 10^{-60}$ cm$^2$ per nucleon -- roughly 30 orders of magnitude below the best direct detection limits. See `mscf_remnant_detection/` for the full analysis.

In [ ]:
# Remnant derivation chain (Eqs. 38-43)
k_B = 1.380649e-23
M_P = np.sqrt(hbar * c_light / G)

# Eq. 38: kappa_max
kappa_max = c_light**2 / (2 * l_P)

# Eq. 39: M_min = M_P / 2
M_min = M_P / 2

# Eq. 43: T_eff = T_H [1 - (1/4)(M_P/M)^2]
def T_eff(M):
    T_H = hbar * c_light**3 / (8 * np.pi * G * M * k_B)
    return T_H * (1 - 0.25 * (M_P / M)**2)

# Verify T_eff(M_P/2) = 0
correction_at_Mmin = 1 - 0.25 * (M_P / M_min)**2

print('Remnant Properties (MSCF v2.1.7, Section XII)')
print(f'  M_P          = {M_P:.6e} kg')
print(f'  M_min = M_P/2 = {M_min:.6e} kg = {M_min / 1.78266192e-27:.3e} GeV')
print(f'  kappa_max    = {kappa_max:.6e} m/s^2')
print(f'  r_s(M_min)   = {2*G*M_min/c_light**2:.3e} m = {2*G*M_min/(c_light**2 * l_P):.6f} l_P')
print()
print(f'  T_eff at various masses:')
for factor in [100, 10, 2, 1, 0.5]:
    M = factor * M_P
    T = T_eff(M)
    print(f'    M = {factor:6.1f} M_P:  T_eff = {T:.3e} K')
print()
print(f'  Critical check: correction at M_P/2 = {correction_at_Mmin:.2e} (should be 0)')
print(f'  Gate 0 PASS: {abs(correction_at_Mmin) < 1e-15}')

---

All results above are consistent with MSCF v2.1.7. The cosmological sector shows null departures from LCDM due to scale separation. The primary falsifiable prediction is the gravitational wave echo signature (Sections XI-XIII).